## Какие параметры выучивают свертки?

Ранние слои CNN выучивают низкоуровневые признаки (края, Gabor-фильтры), средние — текстуры и повторяющиеся паттерны, глубокие — более сложные и класс-специфичные признаки. Но «сложные» не обязательно значит «форма объекта» — стандартные CNN часто опираются на сложные текстурные комбинации, а не на глобальную форму, что отличает их от человеческого восприятия.

В [**блоге**](https://deeplizard.com/learn/video/cNBBNAxC8l4) описан прикольный метод для визуализации того, какой сигнал (какая картинка) максимизирует значение одного фильтра свертки.

## Интерпретация предсказаний сверточных классификаторов

| | **Saliency Maps** | **Grad-CAM** | **LRP** |
| --- | --- | --- | --- |
| **💡 Идея** | Градиент выхода по пикселям | Градиенты + карты признаков последнего conv | Распространение релевантности по слоям. Сохранение общей суммы релевантности (этого нет в градиентных методах) |
| **📐 Результат** | Пиксельная карта, часто шумная | Крупные регионы, класс-специфично | Пиксельная релевантность |
| **🔧 Модели** | Любая DNN | Только CNN | Любые (нужны правила по слоям) |
| **⚡ Сложность** | Низкая (один backprop) | Средняя | Высокая |
| **⚠️ Минусы** | Шум, не класс-специфично | Низкое разрешение | Сложная настройка правил |


### Class Activation Map

Интерпретация предсказаний сверточных нейросетей

http://cnnlocalization.csail.mit.edu/



<img src="static/CAM.png" width=400 />

Пояснения к картинке:

1. Серые квадраты -- это активации (тензоры) после применения сверток на разных слоях
2. Сине-красно-зеленый квадрат -- это последняя активация, перед `GAP`
3. `GAP` -- Global Average Polling
4. w1, w2, ... -- веса полносвязной сети


In [ ]:
!wget https://raw.githubusercontent.com/m12sl/dl-hse-2021/main/05-computer-vision-1/imagenet_1k.json

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import json

from tqdm.auto import tqdm

with open("imagenet_1k.json") as fin:
    clsidx = json.load(fin)


In [ ]:
net = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
net.layer4, net.avgpool, net.fc  # 3 last layers

In [ ]:
!wget https://horseyhooves.com/wp-content/uploads/2021/09/Brown-horse-and-Yorkshire-Terrier-dog.jpg.webp

In [ ]:
IMAGE_PATH = "./Brown-horse-and-Yorkshire-Terrier-dog.jpg.webp"


def load_image(path: str, size: tuple[int, int] = (300, 210)) -> torch.Tensor:
    """Загружает изображение, нормализует по ImageNet и возвращает тензор [1, 3, H, W]."""
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    img = img.astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 1, 3)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 1, 3)
    img = (img - mean) / std
    img = img.astype(np.float32)
    img = np.transpose(img, [2, 0, 1])[None, ...]
    return torch.tensor(img)


img = load_image(IMAGE_PATH)

with torch.no_grad():
    net.eval()
    out = net(img)
    print(out.argmax(dim=-1))

In [ ]:
img.size()

In [ ]:
values, idx = out.topk(5, dim=-1)
idx = idx.numpy().reshape(-1)
print("values", values)
print("idx.  ", idx)

In [ ]:
[  clsidx[str(l)] for l in idx ]

In [ ]:
net = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)

feature_maps: list[torch.Tensor] = []


def hook_fn(module: nn.Module, inp: tuple, out: torch.Tensor) -> None:
    """Forward-хук: сохраняет вход avgpool (последние feature maps)."""
    feature_maps.append(inp[0])


with torch.no_grad():
    net.avgpool.register_forward_hook(hook_fn)
    net.eval()
    out = net(img).numpy()

x = feature_maps[0]
print("Feature maps shape:", x.shape)

with torch.no_grad():
    fc = net.fc
    # Применяем fc к каждой пространственной позиции
    xx = x.permute([0, 2, 3, 1])
    print("Permuted shape:", xx.shape)
    y = fc(xx)
    print("Per-pixel logits shape:", y.shape)
    y = y.numpy()[0, ...]

In [ ]:
import seaborn as sns

original = cv2.imread(IMAGE_PATH)
# plt.imshow(original)
# print(original.shape)

for i in idx:
    print(i, clsidx[str(i)])
    plt.figure()
    cam = y[..., i]
    cam /= (cam.max() + 1e-5)
    # cam = cam.transpose()
    # print(cam.shape)
    cam = np.absolute(cam)
    cam = cv2.resize(cam, (671, 404))
    t = original.astype(np.float32) / 255.0
    t[..., 1] = cam
    t[..., 2] = cam
    plt.imshow(t)

    plt.show()